# 두 번째 베이스라인

스포츠, 사회, 과학, 정치, 종교 등 20개의 카테고리의 토픽을 분류하는

영어 뉴스 데이터를 통해 뉴스 그룹을 분류하는 대회입니다.

이번 베이스라인에서는 TF-IDF를 활용해 문장 벡터를 만들기 위한

TfidfVectorizer 을 적용하여 모델의 성능을 높여보겠습니다.

베이스라인을 통해 자연어 처리 기초에 입문해보세요!

## 데이터 불러오기

In [4]:
#csv 형식의 training 데이터를 로드합니다.
import pandas as pd 

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

#데이터 살펴보기 위해 데이터 최상단의 5줄을 표시합니다.
train.head() 

,id,text,target
0,0,"\nThey were, and even if Washington might cons...",10
1,1,"We run ""SpaceNews & Views"" on our STAREACH BBS...",14
2,2,\n\n\nNot to worry. The Masons have been demo...,19
3,3,"Only Brendan McKay, or maybe ARF, would come t...",17
4,4,Help: I am running some sample problems from O...,5


# 결측치 확인

결측치(NA: Not Available)란 값이 누락된 데이터를 말합니다.

보다 정확한 분석을 하기 위해서는 데이터의 결측치를 확인하고 적절히 처리해주어야 합니다.

이번 데이터에 결측치가 있나 확인해볼까요?

In [5]:
def check_missing_col(dataframe):
    missing_col = []
    for col in dataframe.columns:
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            print(f'결측치가 있는 컬럼은: {col} 입니다')
            print(f'해당 컬럼에 총 {missing_values} 개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if missing_col == []:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(train)

결측치가 존재하지 않습니다


이제 본격적으로 모델을 설계하기 위해 데이터를 문서와 label 로 나누어 줍니다

In [6]:
X = train.text #training 데이터에서 문서 추출
y = train.target #training 데이터에서 라벨 추출

In [7]:
X.head() #데이터 살펴보기

0    \nThey were, and even if Washington might cons...
1    We run "SpaceNews & Views" on our STAREACH BBS...
2    \n\n\nNot to worry.  The Masons have been demo...
3    Only Brendan McKay, or maybe ARF, would come t...
4    Help: I am running some sample problems from O...
Name: text, dtype: object

In [8]:
y.head() #데이터 살펴보기

0    10
1    14
2    19
3    17
4     5
Name: target, dtype: int64

# Text 기초 전처리

EDA 파트에서 했던 것 처럼 Text를 전처리 해보겠습니다.

In [9]:
train.text

0       \nThey were, and even if Washington might cons...
1       We run "SpaceNews & Views" on our STAREACH BBS...
2       \n\n\nNot to worry.  The Masons have been demo...
3       Only Brendan McKay, or maybe ARF, would come t...
4       Help: I am running some sample problems from O...
                              ...                        
9228    \n\nPrecisely, why not Cuba??  Why not???  The...
9229    Your Custom Resume On Disk!\n \n              ...
9230    Throughout the years of the Israel/Arab-Palest...
9231    Does anyone know if there are any devices avai...
9232    \n\n      Give ME a break, chum.  Are you tell...
Name: text, Length: 9233, dtype: object

위와 같이 text 내용을 보면 \n, & 등 불필요한 문자가 들어가 있습니다.

정규표현식을 이용해 이를 전처리하여 깔끔한 문장을 만들어 보도록 해보도록 하겠습니다.

In [10]:
import re 

def clean_text(texts): 
    corpus = [] 
    for i in range(0, len(texts)): 

        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"\n\]\[\>\<]', '',texts[i]) #@%*=()/+ 와 같은 문장부호 제거
        review = re.sub(r'\d+','', review)#숫자 제거
        review = review.lower() #소문자 변환
        review = re.sub(r'\s+', ' ', review) #extra space 제거
        review = re.sub(r'<[^>]+>','',review) #Html tags 제거
        review = re.sub(r'\s+', ' ', review) #spaces 제거
        review = re.sub(r"^\s+", '', review) #space from start 제거
        review = re.sub(r'\s+$', '', review) #space from the end 제거
        review = re.sub(r'_', ' ', review) #space from the end 제거
        corpus.append(review) 
        
    return corpus


In [11]:
temp = clean_text(train['text']) #메소드 적용
train['text'] = temp
print(train)

temp = clean_text(test['text']) #test셋에서도 똑같은 전처리 과정을 해줍니다.
test['text'] = temp

        id                                               text  target
0        0  they were and even if washington might conside...      10
1        1  we run spacenews views on our stareach bbs a l...      14
2        2  not to worry the masons have been demonized an...      19
3        3  only brendan mckay or maybe arf would come to ...      17
4        4  help i am running some sample problems from or...       5
...    ...                                                ...     ...
9228  9228  precisely why not cuba why not the hatians are...      17
9229  9229  your custom resume on disk macintosh or ibm co...       6
9230  9230  throughout the years of the israelarabpalestin...      17
9231  9231  does anyone know if there are any devices avai...       4
9232  9232  give me a break chum are you telling me that c...      18

[9233 rows x 3 columns]


위의 데이터프레임과 같이 text가 깔끔히 정리되었습니다!

## 토큰화

토큰화(Tokenization)란 말뭉치(Corpus)를 주어진 단위(Token)로 나누는 과정을 의미합니다.

따라서 다음과 같이 크게 두 종류로 구분해볼 수 있습니다.

문장 토큰화
단어 토큰화
먼저 말뭉치를 문장 단위로 나누는 문장 토큰화(sentence tokenization)의 예시를 살펴보겠습니다.

In [12]:
text = "Hello, nice to meet you. What's your name? Have a nice day! See you soon." # 예시 문장을 정의합니다.

영어에 대한 전처리는 대표적으로 nltk 패키지를 사용합니다.   
이를 이용하기 위한 라이브러리를 다운받아 주겠습니다.

In [ ]:
import nltk
from nltk.corpus import stopwords
from os import path
nltk.download('all')

In [14]:
from nltk.tokenize import sent_tokenize

print('문장 토큰화 결과 ==>',sent_tokenize(text))

문장 토큰화 결과 ==> ['Hello, nice to meet you.', "What's your name?", 'Have a nice day!', 'See you soon.']


따로 분리 기준으로 "!", "?", "." 등을 설정할 필요 없이 문장 단위로 말뭉치가 분리됩니다!

다음으로 단어 단위로 나누는 단어 토큰화(word tokenization)의 예시를 살펴보겠습니다.

In [15]:
from nltk.tokenize import word_tokenize

print('단어 토큰화 결과 ==>', word_tokenize(text))

단어 토큰화 결과 ==> ['Hello', ',', 'nice', 'to', 'meet', 'you', '.', 'What', "'s", 'your', 'name', '?', 'Have', 'a', 'nice', 'day', '!', 'See', 'you', 'soon', '.']


단어 단위로 깔끔하게 분리되네요. "!", "." 등의 구두점도 하나의 단어로 취급되는 것을 확인할 수 있습니다.

이렇게 토큰화에 대해서 간단히 살펴보았습니다.

그럼 뉴스 그룹 데이터에서도 같은 방법으로 토큰화를 진행시켜  
'tokenized_stem' 열에 새로 토큰화 된 데이터를 생성해보겠습니다.

In [16]:
tokenized = [] # 데이터프레임의 한 컬럼으로 추가할 리스트
for sentence in train['text']: # 전처리된 리뷰들을 하나씩 꺼내옵니다
    tokens = nltk.word_tokenize(sentence)
    tokenize = " ".join(tokens) # tokens라는 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    tokenized.append(tokenize) # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가해줍니다
train["tokenized_stem"] = pd.DataFrame(tokenized) # 리스트를 데이터프레임으로 변환해 tokenized_stem라는 컬럼명으로 추가해줍니다.

train.head() # 데이터 확인

,id,text,target,tokenized_stem
0,0,they were and even if washington might conside...,10,they were and even if washington might conside...
1,1,we run spacenews views on our stareach bbs a l...,14,we run spacenews views on our stareach bbs a l...
2,2,not to worry the masons have been demonized an...,19,not to worry the masons have been demonized an...
3,3,only brendan mckay or maybe arf would come to ...,17,only brendan mckay or maybe arf would come to ...
4,4,help i am running some sample problems from or...,5,help i am running some sample problems from or...


## 품사 태깅(POS Tagging)

자 이제 다음 단계인 품사 태깅에 대해 알아봅시다.

품사 태깅이란 주어진 텍스트를 형태소 단위로 나눈 뒤, 각 형태소에 해당 품사를 태깅하여 리스트화 하는 것입니다.

그럼 토큰화된 단어의 품사를 nltk의 pos_tag메소드를 이용하여 분리해보도록 하겠습니다.

In [17]:
# pos_tag()의 입력값으로는 단어의 리스트가 들어가야 한다.
print(nltk.pos_tag(nltk.word_tokenize(train['text'][4])))

[('help', 'NN'), ('i', 'VB'), ('am', 'VBP'), ('running', 'VBG'), ('some', 'DT'), ('sample', 'NN'), ('problems', 'NNS'), ('from', 'IN'), ('oreilly', 'RB'), ('volume', 'NN'), ('xt', 'JJ'), ('intrisics', 'NNS'), ('programming', 'VBG'), ('manual', 'JJ'), ('chapter', 'NN'), ('popupdialog', 'NN'), ('boxes', 'NNS'), ('and', 'CC'), ('so', 'RB'), ('onin', 'JJ'), ('example', 'NN'), ('page', 'NN'), ('creating', 'VBG'), ('a', 'DT'), ('popup', 'NN'), ('dialog', 'NN'), ('boxthe', 'JJ'), ('application', 'NN'), ('creates', 'VBZ'), ('window', 'VBP'), ('with', 'IN'), ('a', 'DT'), ('button', 'NN'), ('quit', 'NN'), ('and', 'CC'), ('press', 'NN'), ('methe', 'VBP'), ('button', 'NN'), ('press', 'NN'), ('me', 'PRP'), ('pops', 'VBZ'), ('up', 'RP'), ('a', 'DT'), ('dialog', 'NN'), ('box', 'IN'), ('the', 'DT'), ('strange', 'JJ'), ('feature', 'NN'), ('ofthis', 'JJ'), ('program', 'NN'), ('is', 'VBZ'), ('that', 'IN'), ('it', 'PRP'), ('always', 'RB'), ('pops', 'VBZ'), ('up', 'RP'), ('the', 'DT'), ('dialog', 'NN'), ('

품사 태깅은 꼭 필요한 품사(ex. 명사, 동사)를 추출할 때 유용하게 쓰일 수 있습니다!

뉴스 텍스트는 명사만으로 그룹 분류를 판단하기 어렵기 때문에

우선 임의로 명사, 동사, 형용사, 부사를 추출하여 사용해보겠습니다. (판단에 따라 설정해주세요 !)

In [18]:
def postagging(dataframe):
    main_pos = [] # 데이터프레임의 새 컬럼이 될 리스트
    for sentence in dataframe['text']: # 리뷰들을 하나씩 가져옵니다
        pos = nltk.pos_tag(nltk.word_tokenize(sentence)) # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받습니다
        main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ('JJ', 'JJR', 'JJS', #형용사
                                                                         'NN', 'NNS', 'NNP', 'NNPS', #명사
                                                                         'RB', 'RBR', 'RBBS', #부사
                                                                         'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ' #동사
                                                                         )] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가합니다.
        main_words_str = " ".join(main_words) # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
        main_pos.append(main_words_str) # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가해줍니다
        dataframe["main_pos"] = pd.DataFrame(main_pos) # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가해줍니다.

postagging(train)
postagging(test) ## test셋도 똑같이 품사태깅을 적용해줍니다.
train.head() # 데이터 확인

,id,text,target,tokenized_stem,main_pos
0,0,they were and even if washington might conside...,10,they were and even if washington might conside...,were even washington consider patty bust id tr...
1,1,we run spacenews views on our stareach bbs a l...,14,we run spacenews views on our stareach bbs a l...,run spacenews views stareach bbs localoperatio...
2,2,not to worry the masons have been demonized an...,19,not to worry the masons have been demonized an...,not worry masons have been demonized harrassed...
3,3,only brendan mckay or maybe arf would come to ...,17,only brendan mckay or maybe arf would come to ...,only brendan mckay maybe arf come rescue nazir...
4,4,help i am running some sample problems from or...,5,help i am running some sample problems from or...,help i am running sample problems oreilly volu...


자 이제 총 두 개의 전처리된 컬럼이 생성되었습니다!

어떤 전처리 과정이 성능이 가장 잘 나올지는 미지수입니다. 직접 다양한 시도를 통해 최적의 전처리 프로세스를 찾아보세요!

이번 베이스라인에서는 마지막에 생성한 main_pos 컬럼을 이용해 모델 학습을 진행하겠습니다.



# 모델 학습

## train.csv 학습/검증셋 분리하기

모델링 후 성능을 검증하기 위해 학습셋과 검증셋으로 train 데이터를 나누어 보겠습니다.

In [19]:
from sklearn.model_selection import train_test_split

data = train.copy()
train, val = train_test_split(data)
train.reset_index(inplace=True) # 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화해주었습니다.
val.reset_index(inplace=True)

In [20]:
print( 'train 데이터 셋 모양 :', train.shape)
print( 'val 데이터 셋 모양 :', val.shape)

train 데이터 셋 모양 : (6924, 6)
val 데이터 셋 모양 : (2309, 6)


train 셋은 6924개, val 셋은 2309개 데이터로 나뉘어 진 것을 확인할 수 있습니다.

## 벡터화

CountVectorizer를 사용하였습니다.

이 부분에 대한 설명은 [첫번째 베이스라인](https://dacon.io/competitions/official/235884/codeshare/4738?page=1&dtype=recent)에 자세히 나와있으니 참고 바랍니다.

In [21]:
X_train = train.main_pos #training 데이터에서 문서 추출
y_train = train.target #training 데이터에서 라벨 추출

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer() #countvectorizer 생성
vectorizer.fit(X_train) # countvectorizer 학습
X_train_vec = vectorizer.transform(X_train) # transform

## 모델 학습

In [23]:
from sklearn.linear_model import LogisticRegression #모델 불러오기
model = LogisticRegression(max_iter=500) #객체에 모델 할당
model.fit(X_train_vec, y_train) #모델 학습

LogisticRegression(max_iter=500)

## 검증셋으로 모델 성능 검증

트레인 셋에서 학습된 모델을   
검증 셋을 통해서 얼마나 성능이 나오는지 확인해보도록 하겠습니다.

In [24]:
X_val = val.main_pos #validation 데이터에서 전처리된 문서 추출
y_val = val.target #validation 데이터에서 라벨 추출

X_val_vec = vectorizer.transform(X_val) # train셋으로 fit한 벡터라이저 이용해 transform

In [25]:
y_pred = model.predict(X_val_vec)
print(y_pred)

[16 16  2 ... 19  3  7]


In [24]:
from sklearn import metrics
print('Logistic Regression 의 예측 정확도는', round(metrics.accuracy_score(y_val, y_pred),3)) # 정확도 확인

Logistic Regression 의 예측 정확도는 0.605


검증 셋을 예측해본 결과 0.605의 정확도를 얻었습니다!

조금 더 성능을 끌어 올려보는 작업을 해보겠습니다.

# TF-IDF

Tf-Idf (Term Frequency - Inverse Document Frequency)는 

TF(단어 빈도, term frequency) : 특정한 단어가 문서 내에 얼마나 자주 등장하는지를 나타내는 값  
IDF(역문서 빈도, inverse document frequency) : 단어 자체가 문서군 내에서 자주 사용하여 얼마나 그 단어가 흔하게 등장하는지를 나타내는 값의 역수   
를 곱한 값입니다.

즉, 특정 문서 내에서 단어 빈도가 높을 수록, 그리고 전체 문서들 중 그 단어를 포함한 문서가 적을 수록 TF-IDF값이 높아집니다.

특정 단어를 포함하는 문서들이 많을 수록 로그 함수 안의 값이 1에 가까워지게 되고,   
이 경우 IDF값과 TF-IDF값은 0에 가까워지게 되는 것입니다.

따라서 이 값을 이용하면 모든 문서에 흔하게 나타나는 단어를 걸러내는 효과를 얻을 수 있습니다. 

TF-IDF는 주로 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업,  
 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 쓰입니다.

그럼, TF-IDF를 활용해 문장 벡터를 만들기 위한   
TfidfVectorizer 을 적용하여 모델의 성능을 높여보겠습니다!

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

X_train = train.main_pos #training 데이터에서 문서 추출
y_train = train.target #training 데이터에서 라벨 추출

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환. 
tfidf_vect = TfidfVectorizer(ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)

X_val = val.main_pos #validation 데이터에서 전처리된 문서 추출
y_val = val.target #validation 데이터에서 라벨 추출

X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_val_tfidf_vect = tfidf_vect.transform(X_val) # train셋으로 fit한 벡터라이저 이용해 transform
print('학습 & 테스트 데이터 Text의 TfidfVectorizer Shape:',X_train_tfidf_vect.shape, X_val_tfidf_vect.shape)

# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression(solver='liblinear', C = 10) 
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_val_tfidf_vect)
print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(metrics.accuracy_score(y_val ,pred)))

학습 & 테스트 데이터 Text의 TfidfVectorizer Shape: (6924, 630692) (2309, 630692)
TF-IDF Logistic Regression 의 예측 정확도는 0.712


TF-IDF를 적용하여 검증 셋을 예측해본 결과 0.712의 정확도로 무려 0.107 이나 올랐군요 !



## test.csv 분류하기


이제 본격적으로 대회에서 주어진 정답이 없는 test 데이터의 라벨을 예측해보겠습니다.

그럼, 트레인 셋과 검증 셋으로 나누어 성능을 확인해 준 것을 하나의 트레인 셋으로 통합하여

다시 모델을 만들어주겠습니다.

In [31]:
X_train = data.main_pos #전체 training 데이터에서 문서 추출
y_train = data.target #전체 training 데이터에서 라벨 추출

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환. 
tfidf_vect = TfidfVectorizer(ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)

X_train_tfidf_vect = tfidf_vect.transform(X_train)

# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression(solver='liblinear', C = 10) 
lr_clf.fit(X_train_tfidf_vect , y_train)

LogisticRegression(C=10, solver='liblinear')

이제 test셋을 모델에 넣어 예측값을 만들어 줍니다!

In [32]:
X_test = test.main_pos
X_test_vec = tfidf_vect.transform(X_test)
pred_test = lr_clf.predict(X_test_vec)
print(pred_test)

[ 3 16 11 ...  4  1 12]


## dacon 대회에 제출하기

이제 예측한 결과를 submission.csv 파일로 만들어서 대회 페이지에 제출해보도록 합시다.

제출한 뒤 리더보드를 통해 결과를 확인합시다.

In [38]:
# 제출용 sample 파일을 불러옵니다.
submission = pd.read_csv('data/sample_submission.csv')
submission.head()

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [39]:
# 위에서 구한 예측값을 그대로 넣어줍니다.
submission['target'] = pred_test

# 데이터가 잘 들어갔는지 확인합니다.
submission

,id,target
0,0,3
1,1,16
2,2,11
3,3,8
4,4,13
...,...,...
9228,9228,16
9229,9229,1
9230,9230,4
9231,9231,1


submission을 csv 파일로 저장합니다.   
index=False란 추가적인 id를 부여할 필요가 없다는 뜻입니다.    
정확한 채점을 위해 꼭 index=False를 넣어주세요.

In [33]:
submission.to_csv("submission_baseline2.csv", index=False)

이렇게 생성된 submission.csv 파일을 데이콘 대회 페이지에 업로드 & 제출하여 결과를 확인해보세요!

문제를 해결하기 위한 여러분의 방법을 코드 공유 게시판에 공유해주세요

좋아요와 댓글을 합산하여 가장 높은 점수를 얻으신 분께 데이콘 후드가 제공됩니다!